## Gridgen visualization

# 1. Load the environement

In [ ]:
import xarray as xr
import plotly.express as px
import os, fnmatch
from datetime import datetime, timedelta
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import numpy as np
import scipy as sp
import pandas as pd
import seaborn as sns
import sys
import holoviews as hv
import geoviews as gv
import geoviews.feature as gf
import plotly.graph_objects as go
import resourcecode
hv.extension('bokeh')
gv.extension('bokeh')

# 2. Set the grid information

In [ ]:
grid={}

In [ ]:
# read meta file - NC-3M
grid['NC-3M']={}
grid['NC-3M']['NAME'] = 'NC-3M'
grid['NC-3M']['PATH'] = '../data/NC-3M/NC100m'
grid['NC-3M']['DX'] =  0.05
grid['NC-3M']['DY'] =  0.05
grid['NC-3M']['LON_WEST'] =  162.4
grid['NC-3M']['LON_EAST'] =  169.3
grid['NC-3M']['LAT_SOUTH'] = -24.6
grid['NC-3M']['LAT_NORTH'] = -17

In [ ]:
# read meta file - GLOB-30M
grid['GLOB-30M']={}
grid['GLOB-30M']['NAME'] = 'GLOB-30M'
grid['GLOB-30M']['PATH'] = '/home/datawork-WW3/CONFIG/GLOBMULTI/GLOBMULTI_ERA5_GLOBCUR_01/data'
grid['GLOB-30M']['DX'] =  0.5
grid['GLOB-30M']['DY'] =  0.5
grid['GLOB-30M']['LON_WEST'] =  -180.
grid['GLOB-30M']['LON_EAST'] =  180.
grid['GLOB-30M']['LAT_SOUTH'] = -78.
grid['GLOB-30M']['LAT_NORTH'] = 83.5

In [ ]:
# read meta file - ATNE-10M
grid['ATNE-10M']={}
grid['ATNE-10M']['NAME'] = 'ATNE-10M'
grid['ATNE-10M']['PATH'] = '/home/datawork-WW3/CONFIG/GLOBMULTI/GLOBMULTI_ERA5_GLOBCUR_01/data'
grid['ATNE-10M']['DX'] =  0.25
grid['ATNE-10M']['DY'] =  1/6
grid['ATNE-10M']['LON_WEST'] =  -30.
grid['ATNE-10M']['LON_EAST'] =  31.25
grid['ATNE-10M']['LAT_SOUTH'] = 25.
grid['ATNE-10M']['LAT_NORTH'] = 73.0001

In [ ]:
# read meta file - GLOB-15M
grid['GLOB-15M']={}
grid['GLOB-15M']['NAME'] = 'GLOB-15M'
grid['GLOB-15M']['PATH'] = '/home/datawork-WW3/CONFIG/TNTM/TNTM_MULTI7M15M_40/data'
grid['GLOB-15M']['DX'] =  0.25
grid['GLOB-15M']['DY'] =  0.25
grid['GLOB-15M']['LON_WEST'] =  -180.
grid['GLOB-15M']['LON_EAST'] =  180.
grid['GLOB-15M']['LAT_SOUTH'] = -78.
grid['GLOB-15M']['LAT_NORTH'] = 80.25

In [ ]:
# read meta file - GLOB-7M
grid['GLOB-7M']={}
grid['GLOB-7M']['NAME'] = 'GLOB-7M'
grid['GLOB-7M']['PATH'] = '/home/datawork-WW3/CONFIG/TNTM/TNTM_MULTI7M15M_40/data'
grid['GLOB-7M']['DX'] =  0.125
grid['GLOB-7M']['DY'] =  0.125
grid['GLOB-7M']['LON_WEST'] =  -180.
grid['GLOB-7M']['LON_EAST'] =  180.
grid['GLOB-7M']['LAT_SOUTH'] = -78.
grid['GLOB-7M']['LAT_NORTH'] = 80.125

# 3. Compute and display the bathy and mask

In [ ]:
for index, gridname in enumerate(grid):
    print(gridname)
    grid[gridname]['LATITUDE']=xr.DataArray(np.arange(grid[gridname]['LAT_SOUTH'],grid[gridname]['LAT_NORTH'],grid[gridname]['DY']))
    grid[gridname]['LONGITUDE']=xr.DataArray(np.arange(grid[gridname]['LON_WEST'],grid[gridname]['LON_EAST'],grid[gridname]['DX']))

In [ ]:
gridname='GLOB-30M'
grid[gridname]['LATITUDE']

In [ ]:
for index, gridname in enumerate(grid):
    print(gridname)
    grid[gridname]['BOT'] = np.loadtxt(grid[gridname]['PATH']+'/'+grid[gridname]['NAME']+'.bot')
    grid[gridname]['MASK'] = np.loadtxt(grid[gridname]['PATH']+'/'+grid[gridname]['NAME']+'.mask')
    grid[gridname]['OBST'] = np.loadtxt(grid[gridname]['PATH']+'/'+grid[gridname]['NAME']+'.obst')

In [ ]:
# split obst into x and y
for index, gridname in enumerate(grid):
    obst=grid[gridname]['OBST']
    obstlen=int(len(obst)/2)
    grid[gridname]['OBSTX']=obst[0:obstlen,]
    grid[gridname]['OBSTY']=obst[obstlen:,]

In [ ]:
# mask dry points
for index, gridname in enumerate(grid):
    bot=grid[gridname]['BOT']
    grid[gridname]['BATHY'] = np.ma.masked_array(bot, bot>0)
    #bathy = np.ma.masked_where(bot==bot.min(), bot)
    #bathy = np.ma.masked_where(mask>1, bathy)

In [ ]:
fig, axs = plt.subplots(1,2, figsize=(15, 5))

for index, gridname in enumerate(grid):

    axs[index].set_title(gridname+' bathy')
    pcm = axs[index].pcolormesh(grid[gridname]['LONGITUDE'],grid[gridname]['LATITUDE'],grid[gridname]['BATHY'])
    plt.colorbar(pcm,ax=axs[index])

In [ ]:
fig, axs = plt.subplots(1,2, figsize=(15, 5))

for index, gridname in enumerate(grid):

    axs[index].set_title(gridname+' mask')
    pcm = axs[index].pcolormesh(grid[gridname]['LONGITUDE'],grid[gridname]['LATITUDE'],grid[gridname]['MASK'],vmin=0,vmax=3)
    plt.colorbar(pcm,ax=axs[index])

In [ ]:
fig, axs = plt.subplots(1,2, figsize=(15, 5))

for index, gridname in enumerate(grid):

    axs[index].set_title(gridname+' obstX')
    pcm = axs[index].pcolormesh(grid[gridname]['LONGITUDE'],grid[gridname]['LATITUDE'],grid[gridname]['OBSTX'],vmin=0,vmax=3)
    plt.colorbar(pcm,ax=axs[index])

In [ ]:
fig, axs = plt.subplots(1,2, figsize=(15, 5))

for index, gridname in enumerate(grid):

    axs[index].set_title(gridname+' obstY')
    pcm = axs[index].pcolormesh(grid[gridname]['LONGITUDE'],grid[gridname]['LATITUDE'],grid[gridname]['OBSTY'],vmin=0,vmax=3)
    plt.colorbar(pcm,ax=axs[index])

# 4. Look at the plot with interactive viewer

In [ ]:
# bathy interactive map
%output size=200
img = hv.Image((grid[gridname]['LONGITUDE'],grid[gridname]['LATITUDE'],grid[gridname]['BATHY'])).opts(colorbar=True,title=gridname+' bathy')
img

In [ ]:
# mask interactive map
gridname='GLOB-15M'
%output size=200
img = hv.Image((grid[gridname]['LONGITUDE'],grid[gridname]['LATITUDE'],grid[gridname]['MASK'])).opts(colorbar=True,title=gridname+' mask',cmap='jet', clim=(0, 3))
img

In [ ]:
# mask interactive map
gridname='GLOB-7M'
%output size=200
img = hv.Image((grid[gridname]['LONGITUDE'],grid[gridname]['LATITUDE'],grid[gridname]['MASK'])).opts(colorbar=True,title=gridname+' mask',cmap='jet', clim=(0, 3))
img

In [ ]:
# obs-x interactive map
%output size=200
img = hv.Image((longitude,latitude,obstX)).opts(colorbar=True,title='obstruction along x',cmap='jet')
img

In [ ]:
# obs-y interactive map
%output size=200
img = hv.Image((longitude,latitude,obstY)).opts(colorbar=True,title='obstruction along y',cmap='jet')
img

# 6. Compute the difference of bathy and mask

In [ ]:
diff_bathy=bathy2-bathy
diff_mask=mask2-mask
diff2_mask = np.ma.masked_array(diff_mask, diff_mask==0)
diff_obstX=obstX2-obstX
diff_obstY=obstY2-obstY

In [ ]:
fig, axs = plt.subplots(1,2, figsize=(10, 5))

axs[0].set_title('difference of bathy gebco2022-NC100m')
pcm = axs[0].pcolormesh(longitude,latitude,diff_bathy,)
plt.colorbar(pcm,ax=axs[0])
axs[1].set_title('difference of mask')
pcm = axs[1].pcolormesh(longitude,latitude,diff_mask,cmap='jet')
plt.colorbar(pcm,ax=axs[1])
fig.tight_layout()

# 7. Remove active boundaries which are not neighbors to a exluded cell

In [ ]:
def find_indices_no_neighbors(arr, target, excluded):
    # Create a mask for the target value
    mask_ta = (arr == target)
    mask_ex = (arr == excluded)

    # Get the shape of the array
    rows, cols = arr.shape

    # List to store the indices
    indices = []

    # Check each cell
    for i in range(rows):
        for j in range(cols):
            if mask_ta[i, j]:  # If the cell is the target value
                # Check neighbors (up, down, left, right)
                neighbors = [
                    (i - 1, j),  # Up
                    (i + 1, j),  # Down
                    (i, j - 1),  # Left
                    (i, j + 1)   # Right
                ]
                
                # Check if any neighbor is equal to the target
                has_neighbor_equal_excluded = False
                has_neighbor_equal_outbound = False
                for ni, nj in neighbors:
                    if ni < 0 or ni >= rows or nj < 0 or nj >= cols:
                        has_neighbor_equal_outbound = True

                for ni, nj in neighbors:
                    if 0 <= ni < rows and 0 <= nj < cols and mask_ex[ni, nj] and not has_neighbor_equal_outbound:
                        has_neighbor_equal_excluded = True
                        break
                
                if not has_neighbor_equal_excluded or has_neighbor_equal_outbound:
                    #indices.append((i, j))
                    arr[i,j] = 1
    
    return arr

In [ ]:
# Example usage
array = np.array([[3, 3, 3, 3],
                  [2, 1, 2, 2],
                  [1, 2, 1, 1],
                  [1, 1, 2, 1]])

target = 2
excluded = 3
result = find_indices_no_neighbors(array, target, excluded)
print(result)

In [ ]:
# remove active boundaries which are not neighbors to a exluded cell
cleanmask = find_indices_no_neighbors(mask, target, excluded)

In [ ]:
# interactive map
%output size=200
img = hv.Image((longitude,latitude,cleanmask)).opts(colorbar=True,title='mask',cmap='jet', clim=(0, 3))
img

In [ ]:
np.savetxt('/home/datawork-WW3/CONFIG/TNTM/TNTM_MULTI7M15M_40/data/GLOB-7M.cleanmask', cleanmask, fmt='%d', delimiter='  ')

# 1020 FORMAT (/' *** WAVEWATCH III ERROR IN WMGLOW : *** '/       &
#         '     CANNOT FIND SOURCE FOR BOUNDARY DATA '/   
# GRID, IX, IY, X, Y:     2    83     2 -0.1698E+03 -0.7788E+02

In [ ]:
# WARNING indexes start at 0 in python and 1 in fortran.
grid['GLOB-7M']['MASK'][1,82]

In [ ]:
mask_parent=grid['GLOB-15M']['MASK']
mask_child=grid['GLOB-7M']['MASK']

lat_parent=grid['GLOB-15M']['LATITUDE']
lat_child=grid['GLOB-7M']['LATITUDE']

lon_parent=grid['GLOB-15M']['LONGITUDE']
lon_child=grid['GLOB-7M']['LONGITUDE']

In [ ]:
(nlat_parent,nlon_parent)=np.shape(mask_parent)
(nlat_child,nlon_child)=np.shape(mask_child)

In [ ]:
mask_child_new=mask_child

for i in range(nlon_child):
    for j in range(nlat_child):
        if mask_child[j,i]==2:
            closest_index_lon_parent = (np.abs(lon_parent - lon_child[i])).argmin().item()
            closest_index_lat_parent = (np.abs(lat_parent - lat_child[j])).argmin().item()
            
            # test if parent mask is sea point
            if mask_parent[closest_index_lat_parent,closest_index_lon_parent]!=1:
                #print(mask_parent[closest_index_lat_parent,closest_index_lon_parent])
                mask_child_new[j,i]=1
                continue
                
            # test if parent mask is at longitude boundary
            if closest_index_lon_parent==0 or closest_index_lon_parent==nlon_parent-1:
                #print('out of longitude boundaries:'+str(closest_index_lon_parent))
                mask_child_new[j,i]=1                
                continue
                
            # test if parent mask is at latitude boundary
            if closest_index_lat_parent==0 or closest_index_lat_parent==nlat_parent-1:
                #print('out of latitude boundaries:'+str(closest_index_lat_parent))
                mask_child_new[j,i]=1                
                continue
                
            # parent mask neighbors
            mask_parent_left=[closest_index_lat_parent,closest_index_lon_parent-1]
            mask_parent_right=[closest_index_lat_parent,closest_index_lon_parent+1]
            mask_parent_up=[closest_index_lat_parent+1,closest_index_lon_parent]
            mask_parent_down=[closest_index_lat_parent-1,closest_index_lon_parent]
            if mask_parent_left!=1 or mask_parent_right!=1 or mask_parent_up!=1 or mask_parent_down!=1:
                #print('parent mask neighbors not all sea points')
                mask_child_new[j,i]=1                
                continue
                
            


In [ ]:
np.savetxt('/home/datawork-WW3/CONFIG/TNTM/TNTM_MULTI7M15M_40/data/GLOB-7M.cleanmask', mask_child_new, fmt='%d', delimiter='  ')